In [100]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import requests as req
import urllib.request as req
import shutil
from contextlib import closing


# Authentication
from configparser import ConfigParser

config = ConfigParser()
config.read("/Users/nathansuberi/Desktop/Code Portfolio/ResourceWatchCode/.env")
# FROM: https://resourcewatch.carto.com/u/wri-rw/your_apps
carto_api_token = config.get("auth", "carto_api_token")

# import seaborn as sns
plt.style.use('ggplot')
% matplotlib inline

In [95]:
# Read the files that are on the FTP
ftp = "ftp://podaac.jpl.nasa.gov/allData/merged_alt/L2/TP_J1_OSTM/global_mean_sea_level/"
df = pd.DataFrame(req.urlopen(ftp).read().splitlines())
df["files"] = df[0].str.split(expand=True)[8].astype(str)
print(df["files"])
df["files"] = df["files"].apply(lambda row: row[2:-1])

# Select the file that contains the data... i.e. ends with .txt, and has "V4" in the name
data_file_index = df["files"].apply(lambda row: row.endswith(".txt") & ("V4" in row))
print(data_file_index)

# Pull out just the file name
remote_file_name = df.loc[data_file_index,"files"].values[0]
remote_file_name

0                  b'GMSL_TPJAOS_V4.jpg'
1    b'GMSL_TPJAOS_V4_199209_201708.txt'
2     b'README_GMSL_folder_contents.txt'
Name: files, dtype: object
0    False
1     True
2    False
Name: files, dtype: bool


'GMSL_TPJAOS_V4_199209_201708.txt'

In [96]:
sea_level = pd.read_csv(ftp+remote_file_name, header = None, sep = '\t')
sea_level

,0
0,HDR Global Mean Sea Level Data
1,HDR
2,HDR This file contains Global Mean Sea Level (...
3,HDR auspices of the NASA MEaSUREs program. The...
4,HDR Climate Research (http://podaac.jpl.nasa.g...
5,"HDR TOPEX/Poseidon, Jason-1 and OSTM/Jason-2 t..."
6,HDR geophysical corrections applied and placed...
7,"HDR time, regardless of the instrument used."
8,HDR
9,HDR The data can be found below. A separate fi...


In [97]:
#delete columns that start with HDR
df = sea_level
df = df[df[0] != 'HDR']
df = df[~df[0].astype(str).str.contains('HDR')]
df = df[~df[0].astype(str).str.contains('999')]
df

,0
44,0 11 1993.0114746 466466 337280.00 ...
45,0 12 1993.0386963 460890 334038.00 ...
46,0 13 1993.0659180 472125 342417.81 ...
47,0 14 1993.0930176 421381 306053.81 ...
48,0 15 1993.1202393 459550 331095.41 ...
49,0 16 1993.1473389 416487 300349.81 ...
50,0 17 1993.1745605 467982 337063.59 ...
51,0 18 1993.2016602 466077 335416.50 ...
52,0 19 1993.2288818 461213 331526.00 ...
54,0 21 1993.2832031 461744 331710.00 ...


In [98]:
# Goal:
# To convert all multi-space breaks in between numbers into single space breaks
test = df.loc[50,0]
test.split()

['0',
 '17',
 '1993.1745605',
 '467982',
 '337063.59',
 '-38.86',
 '96.18',
 '-39.56',
 '-38.82',
 '96.17',
 '-39.52',
 '-37.63']

In [104]:
#create columns if there's a space
pretty=df[0].str.split(expand=True)
#pretty.columns =["altimeter type, 0=dual-frequency  999=single frequency","merged file cycle #","year+fraction of year (mid-cycle)", "number of observations", "number of weighted observations", "GMSL (Global Isostatic Adjustment (GIA) not applied) variation (mm) with respect to TOPEX collinear mean reference","standard deviation of GMSL (GIA not applied) variation estimate (mm) HDR", "smoothed (60-day Gaussian type filter) GMSL (GIA not applied) variation (mm)", "GMSL (Global Isostatic Adjustment (GIA) applied) variation (mm) with respect to TOPEX collinear mean reference", "standard deviation of GMSL (GIA applied) variation estimate (mm)", "smoothed (60-day Gaussian type filter) GMSL (GIA applied) variation (mm)", "smoothed (60-day Gaussian type filter) GMSL (GIA applied) variation (mm); annual and semi-annual signal removed"]
pretty

,0,1,2,3,4,5,6,7,8,9,10,11
44,0,11,1993.0114746,466466,337280.00,-40.28,97.64,-40.64,-40.27,97.64,-40.63,-41.15
45,0,12,1993.0386963,460890,334038.00,-44.20,100.97,-41.90,-44.19,100.96,-41.89,-41.74
46,0,13,1993.0659180,472125,342417.81,-44.17,97.73,-42.04,-44.15,97.72,-42.02,-41.32
47,0,14,1993.0930176,421381,306053.81,-46.32,101.99,-42.38,-46.30,101.97,-42.36,-41.23
48,0,15,1993.1202393,459550,331095.41,-40.91,99.63,-41.60,-40.87,99.62,-41.57,-40.12
49,0,16,1993.1473389,416487,300349.81,-39.32,98.01,-40.54,-39.28,98.00,-40.50,-38.81
50,0,17,1993.1745605,467982,337063.59,-38.86,96.18,-39.56,-38.82,96.17,-39.52,-37.63
51,0,18,1993.2016602,466077,335416.50,-37.62,96.20,-38.85,-37.57,96.18,-38.80,-36.72
52,0,19,1993.2288818,461213,331526.00,-37.80,96.14,-38.45,-37.74,96.11,-38.39,-36.11
54,0,21,1993.2832031,461744,331710.00,-39.55,94.35,-38.49,-39.48,94.32,-38.42,-35.62


In [107]:
import cartoframes
# `base_url`s are of the form `http://{username}.carto.com/` for most users
USERNAME="wri-rw"
cc = cartoframes.CartoContext(base_url='http://{username}.carto.com/'.format(username=USERNAME),
                              api_key=carto_api_token)
cc.write(pretty, 'Global Mean Sea Level Rise')

/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/carto/auth.py:53: UserWarning: You are using unencrypted API key                           authentication!!!
  authentication!!!")
/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/cartoframes/context.py:241: UserWarning: Table will be named `global_mean_sea_level_rise`
  warn('Table will be named `{}`'.format(table_name))


The following columns were changed in the CARTO copy of this dataframe:
0 -> _0
1 -> _1
2 -> _2
3 -> _3
4 -> _4
5 -> _5
6 -> _6
7 -> _7
8 -> _8
9 -> _9
10 -> _10
11 -> _11


KeyError: 'item_queue_id'